In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

# File path to your CSV file
file_path = '/content/drive/MyDrive/data/vessel_movements/vessel_movements_PPT.csv'




# STEP2

In [ ]:
df = pd.read_csv(file_path)

df['timestamp'] = pd.to_datetime(df['timestamp'])

# Sort the DataFrame by 'imo' and 'timestamp'
df = df.sort_values(by=['imo', 'timestamp'])

# Calculate the time difference in seconds between consecutive rows
df['time_diff'] = df.groupby('imo')['timestamp'].diff().dt.total_seconds()

# Identify non-continuous events
# If the time difference is greater than 10,000 seconds, we treat it as a new event
df['is_continuous'] = df['time_diff'] <= 10000

# Handle the non-continuous events by setting the time_diff to NaN or another identifier
df.loc[~df['is_continuous'], 'time_diff'] = None

# STEP 3

In [ ]:
#df.head()
df['vessel_type'].unique()

array(['Yacht', 'Fishing Vessel', 'Passenger Ship', 'Tug',
       'Utility Vessel', 'Refrigerated Cargo Ship', 'Pusher Tug',
       'Cement Carrier', 'Crane Vessel', 'Landing Craft',
       'General Cargo Ship', 'Container Ship (Fully Cellular)',
       'Products Tanker', 'Research Survey Vessel', 'Fish Carrier',
       'Crane Vessel, non propelled', 'Bulk Carrier', 'Bunkering Tanker',
       'Aggregates Carrier', 'Passenger/Ro-Ro Ship (Vehicles)',
       'Crew/Supply Vessel', 'Water Tanker', 'Ro-Ro Cargo Ship',
       'Anchor Handling Vessel', 'Passenger/Cruise', 'Wood Chips Carrier',
       'Chemical Tanker', 'Vehicles Carrier', 'Chemical/Products Tanker',
       'Asphalt/Bitumen Tanker', 'Platform Supply Ship',
       'Open Hatch Cargo Ship', 'Trailing Suction Hopper Dredger',
       'Diving Support Vessel', 'Crude/Oil Products Tanker',
       'Cable Layer', 'Crude Oil Tanker',
       'General Cargo Ship (with Ro-Ro facility)',
       'Anchor Handling Tug Supply', 'Deck Cargo Ship',

In [ ]:
# List of vessel types that qualify as OGVs
ogv_types = [
    'Bulk Carrier',  # Bulk carrier
    'Container Ship (Fully Cellular)',  # Container ship
    'General Cargo Ship',  # General cargo
    'General Cargo Ship (with Ro-Ro facility)',  # General cargo
    'LPG Tanker',  # LPG / Anhydrous ammonia tanker
    'LPG/Chemical Tanker',  # LPG / Anhydrous ammonia tanker
    'Passenger Ship',  # Passenger / Cruise ship
    'Passenger/Cruise',  # Passenger / Cruise ship
    'Passenger/Ro-Ro Ship (Vehicles)',  # Passenger / Cruise ship
    'Products Tanker',  # Product tanker
    'Crude/Oil Products Tanker',  # Product tanker
    'Chemical/Products Tanker',  # Product tanker
    'Asphalt/Bitumen Tanker',  # Product tanker
    'Bunkering Tanker',  # Product tanker
    'Refrigerated Cargo Ship',  # Reefer ship
    'Ro-Ro Cargo Ship',  # Ro-ro ship
    'Vehicles Carrier',  # Ro-ro ship
    'Chemical Tanker',  # Chemical Tankers
    'Chemical/Products Tanker'  # Chemical Tankers
]
    # Add other relevant types as needed

# Create a new column in the DataFrame to indicate if the vessel is an OGV
df['is_ogv'] = df['vessel_type'].isin(ogv_types)

# Print the first few rows to verify
print(df[['vessel_type', 'is_ogv']].head())

        vessel_type  is_ogv
2663186       Yacht   False
2663185       Yacht   False
2658468       Yacht   False
2658467       Yacht   False
2658466       Yacht   False


# STEP 4

In [ ]:
#print(df[df['is_ogv']]['fuel_category'].unique())
#print(df[df['is_ogv']]['date_of_build'].unique())
df['date_of_build'] = pd.to_datetime(df['date_of_build'])

# Function to determine Emissions Tier based on the year
def get_emissions_tier(build_date):
    year = build_date.year
    if year < 2000:
        return 0
    elif 2000 <= year <= 2010:
        return 1
    else:
        return 2

# Apply the function to each row in the DataFrame
# Only apply it to rows where 'is_ogv' is True
df['emissions_tier'] = df[df['is_ogv']].apply(lambda row: get_emissions_tier(row['date_of_build']), axis=1)

# Print the first few rows to verify
print(df[df['is_ogv']][['is_ogv', 'date_of_build', 'emissions_tier']].head())

         is_ogv date_of_build  emissions_tier
2429116    True    2023-06-01             2.0
2429117    True    2023-06-01             2.0
2409788    True    2023-06-01             2.0
2429118    True    2023-06-01             2.0
2429119    True    2023-06-01             2.0


# STEP 5

In [ ]:
print(df['fuel_category'].unique())
#ALL vessels' fuel categories are 2.

[2]


# STEP 6

# STEP 7b

In [ ]:
df['MS']=df['vref']*1.066
#print(df['MS'].head())

# STEP 7c

In [ ]:
df['A']=df[df['is_continuous']]['time_diff']/3600
#print(df[['A','is_continuous']].head())

# STEP 7d

In [ ]:
df['LF']=(df['speed']/df['MS'])**3
#print(df[['speed','LF']].head())

In [ ]:
df.head()

,imo,mmsi,vessel_id,vessel_name,date_of_build,vessel_type,group,timestamp,date_only,lon,...,sfc_ab,ael,abl,time_diff,is_continuous,is_ogv,emissions_tier,MS,A,LF
2663186,1006946,319646000,660707,PARAFFIN,2001-10-01,Yacht,Other,2023-09-30 02:20:00+00:00,2023-09-30,103.783820,...,300.0,NaN,NaN,NaN,False,False,NaN,14.924,NaN,NaN
2663185,1006946,319646000,660707,PARAFFIN,2001-10-01,Yacht,Other,2023-09-30 02:20:41+00:00,2023-09-30,103.786190,...,300.0,NaN,NaN,41.0,True,False,NaN,14.924,0.011389,0.519862
2658468,1006946,319646000,660707,PARAFFIN,2001-10-01,Yacht,Other,2023-09-30 02:25:08+00:00,2023-09-30,103.802060,...,300.0,NaN,NaN,267.0,True,False,NaN,14.924,0.074167,0.519862
2658467,1006946,319646000,660707,PARAFFIN,2001-10-01,Yacht,Other,2023-09-30 02:25:40+00:00,2023-09-30,103.803820,...,300.0,NaN,NaN,32.0,True,False,NaN,14.924,0.008889,0.519862
2658466,1006946,319646000,660707,PARAFFIN,2001-10-01,Yacht,Other,2023-09-30 02:30:08+00:00,2023-09-30,103.818245,...,300.0,NaN,NaN,268.0,True,False,NaN,14.924,0.074444,0.519862


# STEP 7e

In [ ]:
llaf_table =pd.read_csv('/content/drive/MyDrive/data/supplementary_tables/llaf_table.csv')

In [ ]:
# Convert Load percentages to numeric for comparison
#llaf_table['Load'] = llaf_table['Load'].str.rstrip('%').astype('float') / 100
#llaf_table.head()

In [ ]:
df['LLAF_NOx']=0.33*(df['LF'])**(-0.604)
df['LLAF_HC']=0.111*(df['LF'])**(-1.318)
df['LLAF_CO']=0.2037*(df['LF'])**(-0.985)
df['LLAF_PM']=0.2338*(df['LF'])**(-0.812)
df['LLAF_SO2']=0.4167*(df['LF'])**(-0.5)
df['LLAF_CO2']=0.4232*(df['LF'])**(-0.49)
df['LLAF_N2O']=0.33*(df['LF'])**(-0.604)
df['LLAF_CH4']=0.111*(df['LF'])**(-1.318)

In [ ]:
print(df.head(20))

             imo       mmsi  vessel_id vessel_name date_of_build vessel_type  \
2663186  1006946  319646000     660707    PARAFFIN    2001-10-01       Yacht   
2663185  1006946  319646000     660707    PARAFFIN    2001-10-01       Yacht   
2658468  1006946  319646000     660707    PARAFFIN    2001-10-01       Yacht   
2658467  1006946  319646000     660707    PARAFFIN    2001-10-01       Yacht   
2658466  1006946  319646000     660707    PARAFFIN    2001-10-01       Yacht   
2658465  1006946  319646000     660707    PARAFFIN    2001-10-01       Yacht   
1163309  1012763  319095800     678893  CLOUDBREAK    2016-05-01       Yacht   
1163310  1012763  319095800     678893  CLOUDBREAK    2016-05-01       Yacht   
1163311  1012763  319095800     678893  CLOUDBREAK    2016-05-01       Yacht   
1163312  1012763  319095800     678893  CLOUDBREAK    2016-05-01       Yacht   
1163313  1012763  319095800     678893  CLOUDBREAK    2016-05-01       Yacht   
1163314  1012763  319095800     678893  

# STEP 7f

In [ ]:
# Calculate CO2 emissions for the main engine, auxiliary engine, and auxiliary boiler
df['CO2_main_engine'] = df['sfc_me'] * 0.867 * 3.667
df['CO2_aux_engine'] = df['sfc_ae'] * 0.867 * 3.667
df['CO2_aux_boiler'] = df['sfc_ab'] * 0.867 * 3.667

# Assuming 'sfc_me', 'sfc_ae', and 'sfc_ab' are in grams per kWh,
# these calculations will give us the CO2 emission factors in kg of CO2 per kWh.

# Sum the CO2 emissions from the main engine, auxiliary engine, and auxiliary boiler to get the total CO2 emissions
df['total_CO2'] = df['CO2_main_engine'] + df['CO2_aux_engine'] + df['CO2_aux_boiler']
df.head()

,imo,mmsi,vessel_id,vessel_name,date_of_build,vessel_type,group,timestamp,date_only,lon,...,LLAF_CO,LLAF_PM,LLAF_SO2,LLAF_CO2,LLAF_N2O,LLAF_CH4,CO2_main_engine,CO2_aux_engine,CO2_aux_boiler,total_CO2
2663186,1006946,319646000,660707,PARAFFIN,2001-10-01,Yacht,Other,2023-09-30 02:20:00+00:00,2023-09-30,103.783820,...,NaN,NaN,NaN,NaN,NaN,NaN,619.961355,731.23647,953.7867,2304.984525
2663185,1006946,319646000,660707,PARAFFIN,2001-10-01,Yacht,Other,2023-09-30 02:20:41+00:00,2023-09-30,103.786190,...,0.388009,0.397689,0.577936,0.583123,0.489911,0.262895,619.961355,731.23647,953.7867,2304.984525
2658468,1006946,319646000,660707,PARAFFIN,2001-10-01,Yacht,Other,2023-09-30 02:25:08+00:00,2023-09-30,103.802060,...,0.388009,0.397689,0.577936,0.583123,0.489911,0.262895,619.961355,731.23647,953.7867,2304.984525
2658467,1006946,319646000,660707,PARAFFIN,2001-10-01,Yacht,Other,2023-09-30 02:25:40+00:00,2023-09-30,103.803820,...,0.388009,0.397689,0.577936,0.583123,0.489911,0.262895,619.961355,731.23647,953.7867,2304.984525
2658466,1006946,319646000,660707,PARAFFIN,2001-10-01,Yacht,Other,2023-09-30 02:30:08+00:00,2023-09-30,103.818245,...,0.388009,0.397689,0.577936,0.583123,0.489911,0.262895,619.961355,731.23647,953.7867,2304.984525


In [ ]:
filtered_df = df[df['is_ogv']]
fil_df = filtered_df[filtered_df['is_continuous']]

In [ ]:
fil_df.head()

,imo,mmsi,vessel_id,vessel_name,date_of_build,vessel_type,group,timestamp,date_only,lon,...,LLAF_CO,LLAF_PM,LLAF_SO2,LLAF_CO2,LLAF_N2O,LLAF_CH4,CO2_main_engine,CO2_aux_engine,CO2_aux_boiler,total_CO2
2409788,1014620,563195200,978117,MAJESTIC HONOUR,2023-06-01,Passenger Ship,Passenger/Cruise,2023-08-01 10:55:07+00:00,2023-08-01,103.813354,...,1.556023,1.249625,1.169660,1.163635,1.148109,1.686060,619.961355,731.23647,953.7867,2304.984525
2429119,1014620,563195200,978117,MAJESTIC HONOUR,2023-06-01,Passenger Ship,Passenger/Cruise,2023-08-01 13:55:33+00:00,2023-08-01,103.811380,...,3.890901,2.660146,1.862547,1.835793,2.013988,5.747396,619.961355,731.23647,953.7867,2304.984525
2621158,1014620,563195200,978117,MAJESTIC HONOUR,2023-06-01,Passenger Ship,Passenger/Cruise,2023-08-02 00:20:19+00:00,2023-08-02,103.806274,...,2.374996,1.770810,1.449711,1.436066,1.487973,2.968970,619.961355,731.23647,953.7867,2304.984525
2608060,1014620,563195200,978117,MAJESTIC HONOUR,2023-06-01,Passenger Ship,Passenger/Cruise,2023-08-02 05:55:01+00:00,2023-08-02,103.813650,...,1.556023,1.249625,1.169660,1.163635,1.148109,1.686060,619.961355,731.23647,953.7867,2304.984525
2621160,1014620,563195200,978117,MAJESTIC HONOUR,2023-06-01,Passenger Ship,Passenger/Cruise,2023-08-02 05:55:56+00:00,2023-08-02,103.816790,...,1.556023,1.249625,1.169660,1.163635,1.148109,1.686060,619.961355,731.23647,953.7867,2304.984525


In [ ]:
fil_df.fillna(0,inplace=True)

<ipython-input-19-f4e81ee18465>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fil_df.fillna(0,inplace=True)


In [ ]:
fil_df['pro']=fil_df['p']*fil_df['LF']*fil_df['A']*fil_df['CO2_main_engine']*fil_df['LLAF_CO2']

<ipython-input-20-4771295c4f9b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fil_df['pro']=fil_df['p']*fil_df['LF']*fil_df['A']*fil_df['CO2_main_engine']*fil_df['LLAF_CO2']


In [ ]:
fil_df['aux']=fil_df['ael']*fil_df['A']*fil_df['CO2_aux_engine']

<ipython-input-21-270333602ff8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fil_df['aux']=fil_df['ael']*fil_df['A']*fil_df['CO2_aux_engine']


In [ ]:
fil_df['boi']=fil_df['abl']*fil_df['A']*fil_df['CO2_aux_boiler']

<ipython-input-22-f8a323b2d2ae>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fil_df['boi']=fil_df['abl']*fil_df['A']*fil_df['CO2_aux_boiler']


In [ ]:
fil_df.head()

,imo,mmsi,vessel_id,vessel_name,date_of_build,vessel_type,group,timestamp,date_only,lon,...,LLAF_CO2,LLAF_N2O,LLAF_CH4,CO2_main_engine,CO2_aux_engine,CO2_aux_boiler,total_CO2,pro,aux,boi
2409788,1014620,563195200,978117,MAJESTIC HONOUR,2023-06-01,Passenger Ship,Passenger/Cruise,2023-08-01 10:55:07+00:00,2023-08-01,103.813354,...,1.163635,1.148109,1.686060,619.961355,731.23647,953.7867,2304.984525,219.745927,0.0,0.0
2429119,1014620,563195200,978117,MAJESTIC HONOUR,2023-06-01,Passenger Ship,Passenger/Cruise,2023-08-01 13:55:33+00:00,2023-08-01,103.811380,...,1.835793,2.013988,5.747396,619.961355,731.23647,953.7867,2304.984525,1184.906577,0.0,0.0
2621158,1014620,563195200,978117,MAJESTIC HONOUR,2023-06-01,Passenger Ship,Passenger/Cruise,2023-08-02 00:20:19+00:00,2023-08-02,103.806274,...,1.436066,1.487973,2.968970,619.961355,731.23647,953.7867,2304.984525,1000.373558,0.0,0.0
2608060,1014620,563195200,978117,MAJESTIC HONOUR,2023-06-01,Passenger Ship,Passenger/Cruise,2023-08-02 05:55:01+00:00,2023-08-02,103.813650,...,1.163635,1.148109,1.686060,619.961355,731.23647,953.7867,2304.984525,659237.781925,0.0,0.0
2621160,1014620,563195200,978117,MAJESTIC HONOUR,2023-06-01,Passenger Ship,Passenger/Cruise,2023-08-02 05:55:56+00:00,2023-08-02,103.816790,...,1.163635,1.148109,1.686060,619.961355,731.23647,953.7867,2304.984525,4028.675334,0.0,0.0


In [ ]:
print(fil_df['pro'].sum(),fil_df['aux'].sum(),fil_df['boi'].sum())

8186793837.894916 53488364373.37455 40654145227.22592


In [ ]:
def f(row):
    if row['anchorage'] and row['speed']<=1:
        return 'a'
    elif row['speed']>1 and row['nav_stat'] in [0,3,4,8,11,12] and row['maneuvering_zone']:
        return 'm'
    elif row['berth'] and row['speed']<=1:
        return 'h'
    elif row['speed']>1 and row['nav_stat'] in [0,3,4,8,11,12] and not row['maneuvering_zone']:
        return 't'
    else:
        return None


In [ ]:
fil_df['mode']=fil_df.apply(f,axis=1)

<ipython-input-26-6cc5018b2100>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fil_df['mode']=fil_df.apply(f,axis=1)


# last part - electron

In [ ]:
fil_df.head()

,imo,mmsi,vessel_id,vessel_name,date_of_build,vessel_type,group,timestamp,date_only,lon,...,LLAF_N2O,LLAF_CH4,CO2_main_engine,CO2_aux_engine,CO2_aux_boiler,total_CO2,pro,aux,boi,mode
2409788,1014620,563195200,978117,MAJESTIC HONOUR,2023-06-01,Passenger Ship,Passenger/Cruise,2023-08-01 10:55:07+00:00,2023-08-01,103.813354,...,1.148109,1.686060,619.961355,731.23647,953.7867,2304.984525,219.745927,0.0,0.0,t
2429119,1014620,563195200,978117,MAJESTIC HONOUR,2023-06-01,Passenger Ship,Passenger/Cruise,2023-08-01 13:55:33+00:00,2023-08-01,103.811380,...,2.013988,5.747396,619.961355,731.23647,953.7867,2304.984525,1184.906577,0.0,0.0,t
2621158,1014620,563195200,978117,MAJESTIC HONOUR,2023-06-01,Passenger Ship,Passenger/Cruise,2023-08-02 00:20:19+00:00,2023-08-02,103.806274,...,1.487973,2.968970,619.961355,731.23647,953.7867,2304.984525,1000.373558,0.0,0.0,t
2608060,1014620,563195200,978117,MAJESTIC HONOUR,2023-06-01,Passenger Ship,Passenger/Cruise,2023-08-02 05:55:01+00:00,2023-08-02,103.813650,...,1.148109,1.686060,619.961355,731.23647,953.7867,2304.984525,659237.781925,0.0,0.0,t
2621160,1014620,563195200,978117,MAJESTIC HONOUR,2023-06-01,Passenger Ship,Passenger/Cruise,2023-08-02 05:55:56+00:00,2023-08-02,103.816790,...,1.148109,1.686060,619.961355,731.23647,953.7867,2304.984525,4028.675334,0.0,0.0,t


In [ ]:
fil_df['CO2total']=fil_df['pro']+fil_df['aux']+fil_df['boi']

<ipython-input-28-ddb13db2fca0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fil_df['CO2total']=fil_df['pro']+fil_df['aux']+fil_df['boi']


In [ ]:
print(fil_df.groupby(['mode'])['CO2total'].sum())

mode
a    360847714.671
h    174439771.823
m   3989635542.591
t   7097580470.997
Name: CO2total, dtype: float64


In [ ]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [ ]:
fil_df['vessel_id'].unique().size

2257